1494. Parallel Courses II
Hard
Topics
premium lock icon
Companies
Hint
You are given an integer n, which indicates that there are n courses labeled from 1 to n. You are also given an array relations where relations[i] = [prevCoursei, nextCoursei], representing a prerequisite relationship between course prevCoursei and course nextCoursei: course prevCoursei has to be taken before course nextCoursei. Also, you are given the integer k.

In one semester, you can take at most k courses as long as you have taken all the prerequisites in the previous semesters for the courses you are taking.

Return the minimum number of semesters needed to take all courses. The testcases will be generated such that it is possible to take every course.

 

Example 1:


Input: n = 4, relations = [[2,1],[3,1],[1,4]], k = 2
Output: 3
Explanation: The figure above represents the given graph.
In the first semester, you can take courses 2 and 3.
In the second semester, you can take course 1.
In the third semester, you can take course 4.
Example 2:


Input: n = 5, relations = [[2,1],[3,1],[4,1],[1,5]], k = 2
Output: 4
Explanation: The figure above represents the given graph.
In the first semester, you can only take courses 2 and 3 since you cannot take more than two per semester.
In the second semester, you can take course 4.
In the third semester, you can take course 1.
In the fourth semester, you can take course 5.
 

Constraints:

1 <= n <= 15
1 <= k <= n
0 <= relations.length <= n * (n-1) / 2
relations[i].length == 2
1 <= prevCoursei, nextCoursei <= n
prevCoursei != nextCoursei
All the pairs [prevCoursei, nextCoursei] are unique.
The given graph is a directed acyclic graph.

In [ ]:
from typing import List
from collections import deque

class Solution:
    def minNumberOfSemesters(self, n: int, relations: List[List[int]], k: int) -> int:
        prereq = [0] * n
        for u, v in relations:
            prereq[v - 1] |= 1 << (u - 1)
        
        full_mask = (1 << n) - 1
        q = deque([(0, 0)])  # (state, semesters)
        visited = {0}
        
        while q:
            state, semesters = q.popleft()
            if state == full_mask:
                return semesters
            
            available = 0
            for i in range(n):
                if not (state >> i) & 1 and (state & prereq[i]) == prereq[i]:
                    available |= 1 << i
            
            # remove already taken
            available &= ~state
            
            # case 1: take all if <= k
            if available.bit_count() <= k:
                new_state = state | available
                if new_state not in visited:
                    visited.add(new_state)
                    q.append((new_state, semesters + 1))
            else:
                # case 2: try subsets of size k
                subset = available
                while subset:
                    if subset.bit_count() == k:
                        new_state = state | subset
                        if new_state not in visited:
                            visited.add(new_state)
                            q.append((new_state, semesters + 1))
                    subset = (subset - 1) & available

In [ ]:
class Solution:
    def minNumberOfSemesters(self, n: int, relations: List[List[int]], k: int) -> int:
        post = collections.defaultdict(list)
        degree = [0 for _ in range(n)]
        for a, b in relations:
            degree[b - 1] += 1
            post[a - 1].append(b - 1)
        
        @cache
        def dp(mask: int) -> int:
            if mask == (1 << n) - 1: return 0
            take = []
            for i in range(n):
                if not mask & (1 << i) and degree[i] == 0: take.append(i)

            ans = float("inf")
            for k_take in combinations(take, min(k, len(take))):
                new_mask = mask
                for i in k_take:
                    new_mask |= 1 << i
                    for next in post[i]: degree[next] -= 1
                ans  = min(ans, 1 + dp(new_mask))
                for i in k_take:
                    for next in post[i]: degree[next] += 1
            return ans
        
        return dp(0)

In [ ]:
class Solution:
    def minNumberOfSemesters(self, n: int, relations: List[List[int]], k: int) -> int:
        G = [0] * n
        outDegree = [0] * n
        for x, y in relations:
            G[y-1] += 1 << (x-1)
            outDegree[x-1] += 1
        P2 = [1<<y for y in range(n)]
        start = 0
        goal = 2**n-1
        d = collections.deque([(start,0)])
        seen = [0] * (2 ** n)
        while d:
            s, steps = d.popleft()
            available = [y for y in range(n) if (s & G[y] == G[y]) and (s & P2[y] == 0)]
            A1 = [P2[y] for y in available if outDegree[y]] #these courses can unlock others!
            A2 = [P2[y] for y in available if outDegree[y] == 0] 
			# we only take courses in A2 when we have spaces!
            if len(A1) <= k:
                s += sum(A1) + sum(A2[:k-len(A1)])
                if s == goal: return steps + 1
                if not seen[s]:
                    d.append((s,steps+1))
                    seen[s] = 1
            else:
                for batch in itertools.combinations(A1, k):
                    diff = sum(batch)
                    t = s + diff
                    if t == goal: return steps + 1
                    if not seen[t]:
                        d.append((t, steps+1))
                        seen[t] = 1